In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


In [ ]:
IMG_SIZE = 128

# Load metadata
df = pd.read_csv("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv")

# Encode labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['dx'])  # dx contains diagnosis class names
labels = to_categorical(df['label'])

# Image directories
image_dir1 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/"
image_dir2 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2/"

# Load and resize images
images = []
for img_id in df['image_id']:
    path = os.path.join(image_dir1, img_id + ".jpg")
    if not os.path.exists(path):
        path = os.path.join(image_dir2, img_id + ".jpg")
    img = cv2.imread(path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    images.append(img)

images = np.array(images) / 255.0  # Normalize pixel values


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels
)


In [ ]:
def transformer_block(inputs, num_heads=2, ff_dim=128, dropout=0.1):
    # Layer Norm + Multi-Head Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Layer Norm + Feedforward
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Dense(ff_dim, activation='relu')(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(inputs.shape[-1])(x)

    return x + res

def build_model(input_shape=(128, 128, 3), num_classes=7):
    base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = True  # Freeze base model for now

    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs)
    x = layers.Reshape((-1, x.shape[-1]))(x)  # Prepare for transformer (patch sequence)
    x = transformer_block(x)  # One lightweight transformer block
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)


In [ ]:
model = build_model()
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Lower LR for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])


datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

train_gen = datagen.flow(X_train, y_train, batch_size=16)

history = model.fit(
    train_gen,
    validation_data=(X_test, y_test),
    epochs=30,
    verbose=1
)



In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
model.save("skin_cancer_model.h5")


In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import tensorflow.keras.backend as K
import matplotlib.cm as cm

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], 
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


In [ ]:
# Get last conv layer name
for layer in reversed(model.layers):
    if isinstance(layer, tf.keras.layers.Conv2D):
        print("Last Conv Layer:", layer.name)
        break


In [ ]:
def display_gradcam(image, model, label_map, true_label, last_conv_layer_name="Conv_1"):
    img_array = np.expand_dims(image, axis=0)

    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

    # Convert to RGB and resize heatmap to image size
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    heatmap = np.uint8(255 * heatmap)

    heatmap_color = cm.jet(heatmap / 255.0)[:, :, :3]
    heatmap_color = np.uint8(255 * heatmap_color)

    superimposed_img = heatmap_color * 0.4 + image * 255
    superimposed_img = np.uint8(superimposed_img)

    plt.figure(figsize=(6, 6))
    plt.imshow(superimposed_img)
    plt.title(f"True: {label_map[true_label]} | Predicted: {label_map[np.argmax(model.predict(img_array))]}")
    plt.axis('off')
    plt.show()


In [ ]:
# Check layer names inside MobileNetV2
for layer in model.get_layer("mobilenetv2_1.00_128").layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        print(layer.name)


In [ ]:
# Map back label index to class name
label_map = dict(zip(range(len(le.classes_)), le.classes_))

# Pick any image
idx = 10
test_img = X_test[idx]
true_label = np.argmax(y_test[idx])

display_gradcam(test_img, model, label_map, true_label, last_conv_layer_name="mobilenetv2_1.00_128/block_16_project")

